In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

C:\Users\ASUS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
def load_data(path):
    Data = pd.read_csv(path, nrows=100)
    for column in Data.columns:
        Data[column] = Data[column].fillna(method='pad')
        Data[column] = Data[column].fillna(method='bfill')
    return Data
Data = load_data("G:\github项目\Recomendation_system\Data\criteo_sample\criteo_sampled_data.csv")

In [5]:
Data.columns

Index(['label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10',
       'I11', 'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18',
       'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'],
      dtype='object')

In [6]:
def onehotEncoder(Data):
    features = Data.columns
    
    # 对类别特征进行one-hot编码
    for i in range(14, len(features)):
        oh_feature = pd.get_dummies(Data[features[i]], prefix=1)
        Data.drop([features[i]], axis=1, inplace=True)
        Data = pd.concat([Data, oh_feature], axis=1)
    
    # 分出标签和特征
    X = Data.iloc[:, 1:]
    Y = Data.iloc[:, 0]
    Y = Y.replace(0, -1)
    return X, Y
X, Y = onehotEncoder(Data)

In [13]:
class FM(object):
    
    def __init__(self, x, y):
        self.x = np.mat(x)
        self.y = np.array(y)
        self.feaNum = self.x.shape[1]
        self.sampleNum = len(self.y)
    
    def initialize_vector(self, k):
        """初始化参数权重, k为隐向量的长度，含义为隐向量表示特征的多少"""
        
        w = np.ones((self.feaNum, 1))
        
        # 生成n×k的服从0-0.02的正态分布的隐向量矩阵
        v = np.mat(np.zeros((self.feaNum, k)))
        for i in range(self.feaNum):
            for j in range(k):
                v[i, j] = np.random.normal(0, 0.2)
        
        return w, v
    
    def sigmoid(self, x):
        '''定义sigmoid函数'''
        return 1 / (1 + np.exp(-x))
    
    def loss_fn(self, predict, classes):
        '''定义损失函数'''
        m = np.shape(predict)[0]
        cost = []
        error = 0.0
        for i in range(m):
            error += -np.log(self.sigmoid(classes[i] * predict[i]))
            cost.append(error)
        return error
        
    def SGD(self, k, max_iteration, lr):
        '''
        k: 隐向量的维度
        max_iteration: 最大迭代次数
        lr: 学习率
        '''
        m, n = np.shape(self.x)
        w0 = 0
        w, v = self.initialize_vector(k)
        
        for it in range(max_iteration):
            for i in range(m):
                # 计算特征交叉项
                v_1 = self.x[i] * v # 1×k
                v_2 = np.multiply(self.x[i], self.x[i]) * np.multiply(v, v) # 1×k
                
                inte = 0.5 * np.sum(np.multiply(v_1, v_1) - v_2) 
                p = w0 + self.x[i] * w + inte  # 得到预测值

                loss = self.sigmoid(self.y[i] * p[0]) - 1

                w0 = w0 - lr * loss * self.y[i]  # 更新w0
                for j in range(n):
                    if self.x[i, j] != 0:
                        w[i, 0] = w[i, 0] - lr * loss * self.y[i] * self.x[i, j] # 更新w1
                        for t in range(k):
                            # 更新特征隐向量权重
                            v[j, t] *= lr * loss * self.y[i] * (self.x[i, j] * np.sum(np.multiply(v[:, t].T, self.x[i])) - v[j, t] * self.x[i, j] * self.x[i, j])
            if (it + 1) % 10 == 0:
                print('第{}次迭代, 误差为:{}'.format(it+1,
                                           self.loss_fn(self.predict(w0, w, v), self.y)))
        return w0, w, v
        
    def predict(self, w0, w, v):
        m = np.shape(self.x)[0]
        res = []
        for i in range(m):
            
            v_1 = self.x[i] * v
            v_2 = np.multiply(self.x[i], self.x[i]) * np.multiply(v, v)
            inte = 0.5 * np.sum(np.multiply(v_1, v_1) - v_2)
            p = w0 + self.x[i] * w + inte

            pre = self.sigmoid(p[0, 0])

            res.append(pre)
        return res

In [14]:
a = FM(X, Y)

In [ ]:
a.SGD(10, 1000, 0.0001)

<ipython-input-13-052d947b01d7>:24: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


第10次迭代, 误差为:93.90143112436978
第20次迭代, 误差为:86.8399853760753
第30次迭代, 误差为:83.49918276597725
第40次迭代, 误差为:82.39861127389402
第50次迭代, 误差为:80.53826777170113
第60次迭代, 误差为:79.67792425082631
第70次迭代, 误差为:80.43756092934964
第80次迭代, 误差为:79.81758072995149
第90次迭代, 误差为:78.57735171603494
第100次迭代, 误差为:77.33545353820847
第110次迭代, 误差为:76.7170081951601
第120次迭代, 误差为:76.7170081951601
第130次迭代, 误差为:77.09689368820183
第140次迭代, 误差为:77.09689368820183
第150次迭代, 误差为:77.09689368820183
第160次迭代, 误差为:77.09689368820183
第170次迭代, 误差为:77.09689368820183
第180次迭代, 误差为:77.09689368820183
第190次迭代, 误差为:77.09689368820183
第200次迭代, 误差为:77.09717536948814
第210次迭代, 误差为:77.67405034128919
第220次迭代, 误差为:77.23655016732701
第230次迭代, 误差为:76.61643566037095
第240次迭代, 误差为:76.61643566036874
第250次迭代, 误差为:75.99632115341046
第260次迭代, 误差为:75.99632115341046
第270次迭代, 误差为:75.99632115341046
第280次迭代, 误差为:75.99632115341046
第290次迭代, 误差为:75.99632115341046
第300次迭代, 误差为:75.99632115341046
第310次迭代, 误差为:75.99632115341046
第320次迭代, 误差为:75.99632115341046
第330次迭代, 误差为:75.3762